# Module 01: Django Basics & First Project

**Estimated Time:** 1.5 hours  
**Difficulty:** Beginner

---

## Learning Objectives

By the end of this module, you will:

- ✅ Create your first Django project
- ✅ Understand Django project structure
- ✅ Run the Django development server
- ✅ Create your first Django app
- ✅ Understand settings.py configuration

---

## Prerequisites

Before starting this module, make sure you've completed:
- ✅ Module 00: Setup & Introduction
- ✅ Django is installed and verified

## 1. Creating Your First Django Project

Let's create a Django project called **myblog** for our learning journey.

In [ ]:
import os
import subprocess
from pathlib import Path

# Setup paths
notebook_dir = Path.cwd()
projects_dir = notebook_dir.parent / "projects"
project_name = "myblog"
project_path = projects_dir / project_name

print(f"Projects directory: {projects_dir}")
print(f"Project will be created at: {project_path}")

# Create projects directory if it doesn't exist
projects_dir.mkdir(exist_ok=True)

# Change to projects directory
os.chdir(projects_dir)
print(f"\nCurrent directory: {os.getcwd()}")

In [ ]:
# Create Django project
# This runs: django-admin startproject myblog

if not project_path.exists():
    result = subprocess.run(
        ["django-admin", "startproject", project_name],
        capture_output=True,
        text=True,
        cwd=projects_dir,
    )

    if result.returncode == 0:
        print(f"✓ Project '{project_name}' created successfully!")
    else:
        print(f"✗ Error creating project:")
        print(result.stderr)
else:
    print(f"✓ Project '{project_name}' already exists")

## 2. Exploring the Project Structure

Let's see what Django created for us.

In [ ]:
# List project structure
import os


def print_directory_tree(path, prefix="", max_depth=3, current_depth=0):
    """Print directory tree structure"""
    if current_depth >= max_depth:
        return

    try:
        items = sorted(Path(path).iterdir(), key=lambda x: (not x.is_dir(), x.name))

        for i, item in enumerate(items):
            # Skip __pycache__ and .pyc files
            if "__pycache__" in item.name or item.name.endswith(".pyc"):
                continue

            is_last = i == len(items) - 1
            current_prefix = "└── " if is_last else "├── "
            print(f"{prefix}{current_prefix}{item.name}")

            if item.is_dir():
                extension = "    " if is_last else "│   "
                print_directory_tree(item, prefix + extension, max_depth, current_depth + 1)
    except PermissionError:
        pass


print(f"\n{project_name}/")
print_directory_tree(project_path)

### Understanding the Structure

```
myblog/                    ← Outer project folder
├── manage.py              ← Command-line utility for project management
└── myblog/                ← Inner project folder (Python package)
    ├── __init__.py        ← Makes this directory a Python package
    ├── settings.py        ← Project settings and configuration
    ├── urls.py            ← URL routing (main URL configuration)
    ├── asgi.py            ← ASGI config for async deployment
    └── wsgi.py            ← WSGI config for deployment
```

### Key Files Explained

- **manage.py**: Your main tool for running commands
- **settings.py**: All project settings (database, apps, middleware, etc.)
- **urls.py**: Maps URLs to views
- **wsgi.py/asgi.py**: Entry points for web servers

## 3. Examining settings.py

The `settings.py` file is the heart of your Django project. Let's look at key settings.

In [ ]:
# Read and display important parts of settings.py
settings_file = project_path / project_name / "settings.py"

with open(settings_file, "r") as f:
    settings_content = f.read()

# Extract key configurations
print("=" * 60)
print("KEY SETTINGS")
print("=" * 60)

import re

# Find SECRET_KEY
secret_key = re.search(r"SECRET_KEY = ['\"](.*?)['\"]", settings_content)
if secret_key:
    print(f"\n🔑 SECRET_KEY: {secret_key.group(1)[:20]}... (truncated)")

# Find DEBUG
debug = re.search(r"DEBUG = (.*)", settings_content)
if debug:
    print(f"🐛 DEBUG: {debug.group(1)}")

# Find INSTALLED_APPS
print("\n📦 INSTALLED_APPS (default):")
apps_match = re.search(r"INSTALLED_APPS = \[(.*?)\]", settings_content, re.DOTALL)
if apps_match:
    apps = re.findall(r"['\"]([^'\"]+)['\"]", apps_match.group(1))
    for app in apps:
        print(f"  - {app}")

### Important Settings Explained

#### SECRET_KEY
- Cryptographic signing key
- **NEVER share or commit this to version control**
- Used for security features

#### DEBUG
- `True`: Development mode (detailed error pages)
- `False`: Production mode (generic error pages)
- **Must be False in production**

#### INSTALLED_APPS
Default Django apps:
- `django.contrib.admin`: Admin interface
- `django.contrib.auth`: Authentication system
- `django.contrib.contenttypes`: Content type system
- `django.contrib.sessions`: Session framework
- `django.contrib.messages`: Messaging framework
- `django.contrib.staticfiles`: Static file management

## 4. Running the Development Server

Let's start the Django development server!

**Note**: In Jupyter, we'll demonstrate the command. You should run the actual server in a terminal.

In [ ]:
# Check manage.py commands available
os.chdir(project_path)

result = subprocess.run(["python", "manage.py", "help"], capture_output=True, text=True)

print("Available Django management commands:")
print(result.stdout)

### Running the Server (Terminal)

To start the development server, open a **separate terminal** and run:

```bash
cd projects/myblog
python manage.py runserver
```

You should see:
```
Starting development server at http://127.0.0.1:8000/
Quit the server with CTRL-BREAK.
```

Open your browser and visit: **http://127.0.0.1:8000/**

You should see the Django welcome page! 🎉

### Custom Port
To use a different port:
```bash
python manage.py runserver 8080
```

## 5. Running Initial Migrations

Before we can use Django's built-in features (admin, auth), we need to set up the database.

In [ ]:
# Run migrations
result = subprocess.run(
    ["python", "manage.py", "migrate"], capture_output=True, text=True, cwd=project_path
)

print(result.stdout)
if result.returncode == 0:
    print("\n✓ Database migrations completed successfully!")
else:
    print(f"\n✗ Error: {result.stderr}")

### What Just Happened?

Django created a SQLite database (`db.sqlite3`) and set up tables for:
- User authentication
- Sessions
- Admin interface
- Content types
- Permissions

You can see the database file was created:

In [ ]:
# Check if database was created
db_path = project_path / "db.sqlite3"
if db_path.exists():
    size = db_path.stat().st_size
    print(f"✓ Database created: db.sqlite3 ({size:,} bytes)")
else:
    print("✗ Database not found")

## 6. Creating Your First Django App

Remember: A **project** contains **apps**. Let's create a 'blog' app for our blog functionality.

In [ ]:
# Create a blog app
app_name = "blog"
app_path = project_path / app_name

if not app_path.exists():
    result = subprocess.run(
        ["python", "manage.py", "startapp", app_name],
        capture_output=True,
        text=True,
        cwd=project_path,
    )

    if result.returncode == 0:
        print(f"✓ App '{app_name}' created successfully!")
    else:
        print(f"✗ Error: {result.stderr}")
else:
    print(f"✓ App '{app_name}' already exists")

In [ ]:
# View the app structure
print(f"\n{app_name}/")
print_directory_tree(app_path, max_depth=2)

### App Structure Explained

```
blog/
├── __init__.py        ← Makes this a Python package
├── admin.py           ← Register models for admin interface
├── apps.py            ← App configuration
├── models.py          ← Define your data models (database tables)
├── tests.py           ← Write your tests here
├── views.py           ← Handle HTTP requests/responses
└── migrations/        ← Database migration files
    └── __init__.py
```

### Key Files
- **models.py**: Define database structure
- **views.py**: Business logic and request handling
- **admin.py**: Customize admin interface
- **tests.py**: Unit tests
- **migrations/**: Database schema changes

## 7. Registering the App

To use our app, we need to add it to `INSTALLED_APPS` in settings.py.

In [ ]:
# Read current settings
with open(settings_file, "r") as f:
    settings_content = f.read()

# Check if blog app is already registered
if "'blog'" in settings_content or '"blog"' in settings_content:
    print("✓ Blog app is already registered in INSTALLED_APPS")
else:
    # Add blog to INSTALLED_APPS
    # Find the INSTALLED_APPS section and add our app
    settings_content = settings_content.replace(
        "INSTALLED_APPS = [", "INSTALLED_APPS = [\n    'blog',  # Our blog app"
    )

    # Write back to file
    with open(settings_file, "w") as f:
        f.write(settings_content)

    print("✓ Blog app added to INSTALLED_APPS")

# Verify the change
print("\nCurrent INSTALLED_APPS:")
with open(settings_file, "r") as f:
    content = f.read()
    apps_match = re.search(r"INSTALLED_APPS = \[(.*?)\]", content, re.DOTALL)
    if apps_match:
        apps = re.findall(r"['\"]([^'\"]+)['\"]", apps_match.group(1))
        for app in apps:
            print(f"  - {app}")

## 8. Project Status Check

Let's verify everything is set up correctly.

In [ ]:
# Run Django check command
result = subprocess.run(
    ["python", "manage.py", "check"], capture_output=True, text=True, cwd=project_path
)

print(result.stdout)
if "no issues" in result.stdout.lower():
    print("\n✓ Project configuration is valid!")
else:
    print(f"\n⚠ Issues found: {result.stderr}")

## 9. Summary & Next Steps

### What We Accomplished

✅ Created our first Django project (`myblog`)  
✅ Understood the project structure  
✅ Explored `settings.py` configuration  
✅ Ran initial database migrations  
✅ Created our first Django app (`blog`)  
✅ Registered the app in `INSTALLED_APPS`  
✅ Verified the project configuration  

### Project So Far

```
myblog/
├── manage.py
├── db.sqlite3              ← Database
├── myblog/
│   ├── settings.py         ← Updated with 'blog' app
│   ├── urls.py
│   └── ...
└── blog/                   ← Our new app
    ├── models.py
    ├── views.py
    └── ...
```

### What's Next

In **Module 02**, we'll:
- Learn about Django's ORM (Object-Relational Mapping)
- Create our first models (database tables)
- Understand migrations in depth
- Query data using the QuerySet API
- Create relationships between models

### Quick Practice

Before moving on:
1. Start the development server in a terminal
2. Visit http://127.0.0.1:8000/ and see the welcome page
3. Try visiting http://127.0.0.1:8000/admin/ (we'll set this up properly in Module 03)

---

**Great job! Continue to Module 02 to start building your database!** 🚀